In [17]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Classifier with HOG descriptors

In [18]:
from joblib import dump
from sklearn.svm import SVC
import glob

import src.hog as hog
from src.utils import train_test_split, score, read_img, plot_pts

In [19]:
files = [os.path.splitext(os.path.basename(fn))[0] for fn in glob.glob('data/*.jpg')]
pix_per_cell = (16, 16)

### Train a classifier and test its performance

In [20]:
test_files, train_files = train_test_split(files)
len(test_files + train_files) == len(files) and set(test_files) != set(train_files)

True

In [21]:
x_train_tot, y_train_tot, _ = hog.create_hog_dataset(train_files, pix_per_cell)
x_train_tot = np.concatenate(x_train_tot)
y_train_tot = np.concatenate(y_train_tot)
x_train, y_train = hog.select_subset(x_train_tot, y_train_tot)

x_test, y_test, blocks_bounds = hog.create_hog_dataset(test_files, pix_per_cell)

In [22]:
clf = SVC().fit(x_train, y_train)

In [23]:
for i, fn in enumerate(test_files):
    
    img = read_img(f'data/{fn}.jpg')    
    true = pd.read_csv(f'data/{fn}.csv')
    true_pts = list(true.itertuples(name=None, index=False))
    
    pred_pts = hog.predict(clf, x_test[i], blocks_bounds[i], overlap_threshold=2*pix_per_cell[0])

    plt.figure(figsize=(30, 30))

    ax = plt.subplot(121)
    plt.title('Predictions')
    plot_pts(img, pred_pts)

    ax = plt.subplot(122)
    plt.title('Ground truth')
    plot_pts(img, true_pts)

    plt.savefig('plots/' + fn + '_results.png')
    plt.close()

    score(true_pts, pred_pts)
    print()

Precision : 1.000
Recall : 0.682
F1 score : 0.811

Precision : 0.875
Recall : 0.667
F1 score : 0.757

Precision : 0.714
Recall : 0.682
F1 score : 0.698

Precision : 0.938
Recall : 0.714
F1 score : 0.811

Precision : 0.846
Recall : 0.524
F1 score : 0.647

Precision : 0.800
Recall : 0.762
F1 score : 0.780

Precision : 0.909
Recall : 0.500
F1 score : 0.645

Precision : 0.941
Recall : 0.800
F1 score : 0.865

Precision : 0.842
Recall : 0.762
F1 score : 0.800

Precision : 0.882
Recall : 0.714
F1 score : 0.789

Precision : 0.889
Recall : 0.727
F1 score : 0.800



### Train classifier on all the data and save it to disk

In [25]:
x_tot, y_tot, _ = hog.create_hog_dataset(files, pix_per_cell)
x_tot = np.concatenate(x_tot)
y_tot = np.concatenate(y_tot)
x, y = hog.select_subset(x_tot, y_tot)

final_clf = SVC().fit(x, y)
dump(final_clf, 'model/model.joblib') 

['model/model.joblib']